In [1]:
import pandas as pd

prices = pd.read_csv("/content/Nat_Gas.csv")
prices.head()


,Dates,Prices
0,10/31/20,10.1
1,11/30/20,10.3
2,12/31/20,11.0
3,1/31/21,10.9
4,2/28/21,10.9


In [2]:
# Standardize column names
prices.columns = ["date", "price"]

# Convert date column
prices["date"] = pd.to_datetime(prices["date"])

# Add month for seasonality
prices["month"] = prices["date"].dt.month


/tmp/ipython-input-2294950961.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  prices["date"] = pd.to_datetime(prices["date"])


In [3]:
monthly_avg = prices.groupby("month")["price"].mean()
monthly_avg


,price
month,
1,11.775
2,11.700
3,11.775
4,11.175
5,10.785
6,10.700
7,10.900
8,10.825
9,11.075


In [4]:
def get_price_for_date(input_date):
    input_date = pd.to_datetime(input_date)
    month = input_date.month

    # If the date exists in historical data
    if input_date in prices["date"].values:
        return float(prices.loc[prices["date"] == input_date, "price"])

    # Future date: use seasonal (monthly) average
    return float(monthly_avg[month])


In [7]:
def get_price_for_date(input_date):
    input_date = pd.to_datetime(input_date)
    month = input_date.month

    if input_date in prices["date"].values:
        return float(
            prices.loc[prices["date"] == input_date, "price"].iloc[0]
        )

    return float(monthly_avg[month])


In [8]:
get_price_for_date("2022-12-31")
get_price_for_date("2025-01-31")


11.775

The provided monthly natural gas price data was analyzed to identify seasonal trends.
For dates present in the dataset, the historical price is returned.
For future dates, prices are estimated using the average price for the corresponding month to capture seasonality.
This approach provides a simple and interpretable estimate suitable for longer-term storage contract pricing.

In [9]:
def storage_contract_value(
    injection_dates,
    withdrawal_dates,
    volume_per_action,
    max_volume,
    storage_cost_per_month,
    price_function
):
    stored_volume = 0
    total_cost = 0
    total_revenue = 0

    # Buy gas (inject)
    for date in injection_dates:
        price = price_function(date)
        if stored_volume + volume_per_action <= max_volume:
            total_cost += price * volume_per_action
            stored_volume += volume_per_action

    # Sell gas (withdraw)
    for date in withdrawal_dates:
        price = price_function(date)
        if stored_volume >= volume_per_action:
            total_revenue += price * volume_per_action
            stored_volume -= volume_per_action

    # Storage cost
    months_used = len(set(injection_dates + withdrawal_dates))
    storage_cost = months_used * storage_cost_per_month

    return total_revenue - total_cost - storage_cost


In [10]:
injection_dates = ["2021-06-01", "2021-07-01"]
withdrawal_dates = ["2021-12-01", "2022-01-01"]

value = storage_contract_value(
    injection_dates=injection_dates,
    withdrawal_dates=withdrawal_dates,
    volume_per_action=100000,
    max_volume=500000,
    storage_cost_per_month=10000,
    price_function=get_price_for_date
)

print("Contract value:", value)


Contract value: 147500.0


This model prices a natural gas storage contract by calculating
the difference between revenue from selling gas and the cost of
buying gas and storing it. Prices are obtained using the pricing
function from Task 1. Storage costs are assumed to be fixed per
month, and transport delays and interest rates are ignored.
